# Pràctica 2 de PLN: web scraping
En aquesta pràctica realitzarem diferents exercicis sobre captura de continguts web (*web scraping*) usant les llibreries `request` i `BeautifulSoup`.

### Noms:
Introdueix en esta celda els noms dels integrats del grup:\
*Alumne 1* \
*Alumne 2*

In [ ]:
import requests
from bs4 import BeautifulSoup

## Part 1: captura d'una notícia
En aquest apartat ens descarregarem una notícia d'una pàgina web i guardem el seu contingut en memòria.

In [ ]:
url = "https://elpais.com/sociedad/2019/11/13/actualidad/1573632952_315974.html"
try:
    page = requests.get(url)
except:
    print("Error al abrir la URL")

In [ ]:
# parseamos el html usando BeautifulSoup y lo guardamos en la variable `soup`
soup = BeautifulSoup(page.text, 'html.parser')

El contingut de l'article (text del cos) es troba dins d'un `<div>` de la pàgina amb les classes `a_c clearfix` (pots inspeccionar l'estructura HTML de la pàgina en el navegador per a comprovar-lo)

In [ ]:
# Buscamos el <div> correspondiente y sacamos su contenido:
content = soup.find('div', {"class": "a_c clearfix"})

article = []
for i in content.find_all('p'):
    article.append(i.text)
    
print('\n'.join(article))

Ací s'han colat alguns paràgrafs que no pertanyen al cos de la notícia, sinó que estan en seccions interiors. Ho podem evitar usant el paràmetre `recursive=False`. 
A més l'últim paràgraf no forma part del cos de la memòria, el podem filtrar amb `*class_=""` perquè té un atribut de classe específic.

In [ ]:
#optimizando el código
content = soup.find('div', {"class": "a_c clearfix"}).find_all('p', class_="",recursive=False)

article = ('\n').join([i.text for i in content])

print(article)

Si volguérem fer un filtrat més específic de paràgrafs o continguts, podem definir una funció lògica que retorne a `find_all` els elements a considerar:
```python
def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')
```

Que després usem amb:

```python
content.find_all(has_class_but_no_id)
```


In [ ]:
#Por ejemplo los párrafos que contienen la palabra basílica
def contiene_basilica(tag):
    return tag.name=='p' and 'basílica' in tag.text

content = soup.find('div', {"class": "a_c clearfix"})
content.find_all(contiene_basilica)

### Exercici 1
Cerca els paràgrafs dins de l'etiqueta `<div>` amb el cos de la notícia que continguen un element de tipus enllace (`<a>`)

In [ ]:
## Solución


## Part 2: captura de dades meteorològiques
En aquest apartat capturarem dades meteorològiques de la Comunitat Valenciana des de la pàgina de la [AVAMET (Associació valenciana de meteorologia)](https://www.avamet.org).

In [ ]:
#esta página contiene los datos meteorológicos de un día concreto
fecha = '2021-02-10'
r = requests.get("https://www.avamet.org/mx-meteoxarxa.php", params={'id':fecha})

In [ ]:
soup = BeautifulSoup(r.text, "html.parser")

Les dades de tots els municipis de la CV estan en una taula de classe `tDades`

In [ ]:
tabla = soup.find("table", class_="tDades")

Dins de la taula, les dades estan en les files (`<tr>`) que tenen un element`<td class='rEsta'>`. Definim una funció per a filtrar etiquetes amb aquesta classe i busquem tots els elements interns a la taula:

In [ ]:
def clase_rEsta(tag):
    return tag.find(class_="rEsta")

In [ ]:
loc = tabla.find_all(clase_rEsta)

In [ ]:
len(loc)

Ens fixem per exemple en el primer element d'aquesta llista:

In [ ]:
print(loc[0].prettify())

Veiem que algunes de les cel·les d'aquesta fila tenen elements de tipus `<span>`. Si no ens interessen els podríem eliminar amb el mètode `,decompose()` del Tag. Però com el text ens interessa fiquem un espai per a separar el contingut en extraure el text posteriorment amb `.text()`:

In [ ]:
for t in loc:
    for t in t.find_all('span', class_="rEstaDmxo"):
        t.insert_before(' ')
print(loc[0].prettify())

In [ ]:
#Cada celda <td> dentro de la fila es una columna de la tabla
[t.text.strip() for t in loc[0].find_all("td")]

In [ ]:
[t for t in loc[0].find_all("td")]

In [ ]:
#Capturamos toda la tabla
datos = [[t.text.strip() for t in l.find_all("td")] for l in loc]

In [ ]:
#Todas las filas tienen los mismos datos
[len(d) for d in datos]

Ara exportem la taula com un DataFrame de `pandas`:

In [ ]:
import pandas as pd

In [ ]:
data_matrix = pd.DataFrame(datos, columns = ['localidad','Temp','Tmax','Tmin','Humedad','Precip',
                      'Vel.viento','Dir.viento','Vmax_viento']) #Definimos el nombre de las columnas

In [ ]:
data_matrix.info()

In [ ]:
data_matrix.head(5)

També podíem llegir directament la taula en pandas amb el mètode `read_HTML`

In [ ]:
#hace falta instalar la librería 'lxml'
df = pd.read_html(str(tabla))[0] #devuelve una lista de dataframes
df.head(10)

### Exercici 2
Crea un script per a capturar les dades d'un territori i una data concretes a través de la URL:\
`https://www.avamet.org/mx-meteoxarxa.php?id={fecha}&territori={territorio}`\
Els codis de cada territori estan en l'element `select` següent:

In [ ]:
soup.find("select", attrs={'name':"freg_territori"})

In [ ]:
# Solución


### exercici extra:
Captura les dades durant un mes de l'estació de 'València *Camins al Grau' i representa gràficament la seua temperatura mitjana 

In [ ]:
# Solución


## Part 3: dades de la Wikipedia
En aquesta part obtindrem les URL de les entrades en la Wikipedia per a totes les províncies d'Espanya i obtindrem d'elles les seues dades bàsiques en forma de taula.\
El llistat de les províncies d'Espanya es pot descarregar de la pàgina de la Wikipedia següent:\
https://es.wikipedia.org/wiki/Provincia_(España)

In [ ]:
r = requests.get("https://es.wikipedia.org/wiki/Provincia_(España)")
soup = BeautifulSoup(r.text, "html.parser")

In [ ]:
tabla = soup.find("table", class_="wikitable")

In [ ]:
#todas las filas de la tabla (<tr>)
provincias = tabla.find_all("tr")

In [ ]:
#los datos están en la primera celda <td> de cada fila a partir de la 1 (la fila 0 son los encabezados)
print(provincias[1].td.prettify())

In [ ]:
# extraemos el enlace
provincias[1].td.find("a")

In [ ]:
enlaces = []
for p in provincias:
    if p.td:
        ref = p.td.find('a')
        enlaces.append(ref)

In [ ]:
enlaces

In [ ]:
enlaces_df = pd.DataFrame({'provincia': [e.attrs['title'] for e in enlaces],
                           'enlace': [e.attrs['href'] for e in enlaces]})

In [ ]:
enlaces_df

Per exemple, creem el objecte `soup` per a la primera província:

In [ ]:
r = requests.get("https://es.wikipedia.org"+enlaces_df['enlace'][0])
soup = BeautifulSoup(r.text, "html.parser")

Extraurem en un dataframe la informació geogràfica de la taula de la barra lateral dreta (atribut de classe `infobox`):

In [ ]:
tabla = soup.find("table", class_="infobox")

Si inspecciones la seua estructura HTML veuràs una sèrie de tags `tr` de les quals pengen parells de tags `th` i `td` associades. Capturarem els seus textos en dues llistes: `dada` i `valor`, respectivamant.

In [ ]:
dato, valor = [],[]
for t in tabla.find_all("tr"):
    if t.th:
        if t.td:
            dato.append(t.th.text)
            #valor.append(t.td.text) # aparecen caracteres especiales!
            valor.append(' '.join([text for text in t.td.stripped_strings]))

In [ ]:
datos = pd.DataFrame({'Dato': dato, 'Valor': valor})
datos

### Exercici 3
Crea una taula (dataframe) amb la capital, la superfície i la població de cada província d'Espanya.\
Per a trobar en la taula aquestes dades podem fer:

In [ ]:
tabla.find(string='Capital').next.text

In [ ]:
tabla.find(string='Superficie').parent.parent.parent.next_sibling.text

In [ ]:
tabla.find(string='Población').parent.parent.parent.next_sibling.text

Hauràs d'usar expressions regulars per a extraure d'aquests *strings el text buscat. Si la informació no està disponible per a alguna província, emplena la cel·la amb un *string* buit.

In [ ]:
#solución
   